In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import squidpy as sq
from itertools import compress
import networkx as nx

import pyvis as pv

In [2]:
adata_dir = 'adata_env'

adata = sc.read('adata_integrated.h5ad')
patients = adata.obs['patient'].cat.categories

env_names = adata.obs.columns[['env_' in el for el in adata.obs.columns]]
env_names = [e for e in env_names if 'MT' not in e]

rctd_names = adata.obs.columns[['rctd_' in el for el in adata.obs.columns]]
rctd_names = [r for r in rctd_names if 'Mast' not in r]

sigs = rctd_names

In [3]:
# remove outlier samples (mostly normal, strange behavior)
keep = [all([s not in patient for s in ['313']]) for patient in patients]
patients = list(compress(patients, keep))
adata = adata[adata.obs['patient'].isin(patients)]

In [4]:
r=1
Xagg = pd.read_csv('spatial_enrichment_by_regression_r%i_significant_rctd.csv'%r, index_col=[0,1])

In [5]:
# remove neurons and astro
Xagg = Xagg.drop(['rctd_Astro', 'rctd_Neuron'], level=1)
Xagg = Xagg.drop(['rctd_Astro', 'rctd_Neuron'], level=0)

In [6]:
perc_thr = 0.4
Xagg_thr = 0.5

X_agg_pos = Xagg[np.logical_and(Xagg['n_samples_significant']>=perc_thr, Xagg['enrichment']>=Xagg_thr)].copy()

for index in X_agg_pos.index:
    try:
        if X_agg_pos.loc[(index[1],index[0]),'enrichment'] > X_agg_pos.loc[index,'enrichment']:
            X_agg_pos.drop(index, inplace=True)
        else:
            X_agg_pos.drop((index[1],index[0]), inplace=True)
    except KeyError:
        pass

df = X_agg_pos.reset_index()
df = df[df['level_0']!=df['level_1']] # remove self loops

# set max value for enrichment
max_val = 2
df.loc[df['enrichment']>max_val,'enrichment'] = max_val

# add edge to scale others
dummy_val = 3
df = df.append({'level_0':'dummy0','level_1':'dummy1','enrichment':dummy_val,'n_samples_significant':1},\
          ignore_index=True)

df.to_csv('network_graph_edges_rctd_3thr.csv')

/var/folders/rt/bkk9ygc977qblhlsqxs5r77h0000gp/T/ipykernel_19413/1515616545.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'level_0':'dummy0','level_1':'dummy1','enrichment':3,'n_samples_significant':1},\


In [8]:
G = nx.from_pandas_edgelist(df,'level_0','level_1',edge_attr=True)
GEdges = pd.concat([df.set_index(['level_0','level_1']), df.set_index(['level_1','level_0'])]).loc[list(G.edges)]

In [9]:
# adjust plot params
for node in G.nodes:
    if 'Scavenger' in node:
        G.nodes[node]['color'] = 'blue'
    elif 'Complement' in node:
        G.nodes[node]['color'] = 'blue'
    elif 'IL1B' in node:
        G.nodes[node]['color'] = 'red'
    elif 'Inflamm' in node:
        G.nodes[node]['color'] = 'red'
    else:
        G.nodes[node]['color'] = 'gray'
for edge in G.edges:
    G.edges[edge]['value'] = GEdges.loc[edge,'enrichment'].mean()
    G.edges[edge]['color'] = 'gray'

# create vis network
net = pv.network.Network(notebook=True)
rndseed = 4
net.set_options('{"layout": {"randomSeed":%i}}'%rndseed)
# load the networkx graph
net.from_nx(G)
# show
net.show("spatial_enrichment_r%i_rctd_new_thr.html"%r)

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


In [10]:
# create vis network, no labels
net = pv.network.Network(notebook=True, font_color='#10000000', bgcolor='#10000000')
net.set_options('{"layout": {"randomSeed":%i}}'%rndseed)
# load the networkx graph
net.from_nx(G)
# show
net.show("spatial_enrichment_r%i_rctd_new_thr_no_labels.html"%r)

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
